In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
# 定义参数
region = 'daqing'
year = '2015'
startDate = year + '-01-01'
endDate = year + '-12-31'

In [4]:
# # 大庆市
roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,9)

In [5]:
region = 'daqing'
elevation = ee.Image("users/311605001111/hillshade_" + region)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa').toInt()
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

In [6]:
## 指数的计算
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('(B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7)/10000',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

In [7]:
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
        .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
        .filterBounds(roi) \
        .filterDate('2015-05-25','2015-05-31')
# l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
#        .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
#        .filterBounds(roi) \
#        .filterDate('2013-01-01', '2020-12-31') 

In [8]:
def near_orbit(imgCol):
    imgCol_date = imgCol.map(lambda i:i.set({'date': ee.String(i.get('system:id')).slice(-8,)}))
    dates = ee.List(imgCol_date.distinct('date').aggregate_array('date'))
    imgCol_median = ee.ImageCollection(dates.map(lambda i:imgCol_date.filter(ee.Filter.eq('date',i)).median().copyProperties(imgCol_date.filter(ee.Filter.eq('date',i)).first())))
    return imgCol_median

In [ ]:
print(l8.size().getInfo())
print(l8.aggregate_array("system:id").getInfo())
za = near_orbit(l8.map(maskSR))
print(za.size().getInfo())
print(za.aggregate_array('date').getInfo())

In [ ]:
print(za.first().getInfo())
print(za.first().get('SOLAR_AZIMUTH_ANGLE').getInfo())
print(za.first().geometry().getInfo())

In [9]:
za1 = near_orbit(l8.map(water_index)).map(maskSR)
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
Map.addLayer(za1.first().clip(roi), visParams,'za1')
zaza1 = near_orbit(l8)
Map.addLayer(zaza1.first().clip(roi), visParams,'zaza1')

In [10]:
za = near_orbit(l8.map(maskSR).map(water_index))
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
Map.addLayer(za.first().clip(roi), visParams,'za')
zaza = near_orbit(l8)
Map.addLayer(zaza.first().clip(roi), visParams,'zaza')

In [ ]:
region = 'daqing'
sample_region = 'DAQING'
filtered_samples = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_9920_v2')
print(filtered_samples.first().getInfo())

In [ ]:
validPixel = ee.ImageCollection(za).count().select('B3').clip(roi).rename('count')
Map.addLayer(validPixel, {'palette':['white','blue'],'min':0,'max':3},'validPixel')

In [ ]:
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate('2015-05-20','2015-05-31')
l8_dates = l8.map(lambda i:i.set({'date': ee.String(i.get('system:id')).slice(-8,)}))
dates = ee.List(l8_dates.distinct('date').aggregate_array('date'))
filter_images = ee.ImageCollection(dates.map(lambda i:l8_dates.filter(ee.Filter.eq('date',i)).median().copyProperties(l8_dates.filter(ee.Filter.eq('date',i)).first())))
# print(filter_images.first().getInfo())
print(filter_images.first().get('SOLAR_AZIMUTH_ANGLE').getInfo())
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
Map.addLayer(filter_images.first().clip(roi), visParams,'a')

In [ ]:
q = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_119027_20150522')
p = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_119028_20150522')
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
Map.addLayer(q.clip(roi), visParams,'q')
Map.addLayer(p.clip(roi), visParams,'p')
print(q.get('SOLAR_AZIMUTH_ANGLE').getInfo())
print(p.get('SOLAR_AZIMUTH_ANGLE').getInfo())
print(q.get('SOLAR_ZENITH_ANGLE').getInfo())
print(p.get('SOLAR_ZENITH_ANGLE').getInfo())

In [ ]:
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate('2015-05-20','2015-05-31')
l8_dates = l8.map(lambda i:i.set({'date': ee.String(i.get('system:id')).slice(-8,)}))
dates = ee.List(l8_dates.distinct('date').aggregate_array('date'))
filter_images = ee.ImageCollection(dates.map(lambda i:l8_dates.filter(ee.Filter.eq('date',i)).median()))

visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
Map.addLayer(filter_images.first().clip(roi), visParams,'a')
print(filter_images.size().getInfo())

In [ ]:
 'LANDSAT/LC08/C01/T1_SR/LC08_119028_20150506', 

 'LANDSAT/LC08/C01/T1_SR/LC08_120027_20150513',
 'LANDSAT/LC08/C01/T1_SR/LC08_120028_20150513',

 'LANDSAT/LC08/C01/T1_SR/LC08_118028_20150515', 

 'LANDSAT/LC08/C01/T1_SR/LC08_119027_20150522', 
 'LANDSAT/LC08/C01/T1_SR/LC08_119028_20150522', 
 
 'LANDSAT/LC08/C01/T1_SR/LC08_120027_20150529', 
 'LANDSAT/LC08/C01/T1_SR/LC08_120028_20150529'